Test shipping

In [ ]:
import sys
import os
import json
import logging
from kafka import KafkaConsumer, KafkaProducer
from json import loads
import random
import datetime
import mysql.connector
import boto3
from logging.handlers import RotatingFileHandler

In [ ]:
def shipping_consumer():
    # MySQL connection
    mysql_conn = mysql.connector.connect(
        host=MYSQL_HOST,
        user=MYSQL_USER,
        password=MYSQL_PASSWORD,
        database=MYSQL_DATABASE
    )
    mysql_cursor = mysql_conn.cursor()

    # Kafka
    sales_consumer = KafkaConsumer(
        'invoices',
        bootstrap_servers=['localhost:9092'],
        auto_offset_reset='earliest',  # Start consuming from the earliest offset
        enable_auto_commit=True,       # Automatically commit offsets
        group_id='oaken_shipping_group',  # Specify a consumer group
        value_deserializer=lambda x: loads(x.decode('utf-8')))

    sales_consumer.subscribe(topics='invoices')

    shipping_producer = KafkaProducer(bootstrap_servers=['localhost:9092'],,
                            value_serializer=lambda x: json.dumps(x).encode('utf-8'))

    for message in sales_consumer:
        try:
            data = message.value
            invoice = data.get('Invoice', '')
            date_str = data.get('SaleDate')
            sales_date = datetime.datetime.strptime(date_str, '%m/%d/%Y').date()
            sales = data.get('SaleDollars')

            shipping_cost = round(float(sales) * 0.05,2)

            random_days = random.randint(0, 4)
            shipping_date = sales_date + datetime.timedelta(days=random_days)

            # MySQL
            UPDATE_QUERY = """
                UPDATE sales
                SET ShippingDate = %s, ShippingCost = %s
                WHERE Invoice = %s
            """

            update_data = (shipping_date, shipping_cost, invoice)
            mysql_cursor.execute(UPDATE_QUERY, update_data)

            mysql_conn.commit()

            # Kafka topic
            shipping_info = {
                'Invoice': invoice,
                'SalesDate': date_str,
                'SaleDollars': sales,
                'ShippingDate': shipping_date,
                'ShippingCost': shipping_cost
            }

            shipping_producer.send(SHIPPING_TOPIC, value=shipping_info)
            shipping_producer.flush()

        except Exception as e:
            logger.error(f"Error processing message: {e}")
            pass

    # Close MySQL connection
    mysql_cursor.close()
    mysql_conn.close()

In [ ]:
shipping_consumer()

In [ ]:
# sample output
'''
{'Invoice/Item Number': 'S29198800001', 'Date': '11/20/2015', 'Store Number': '2191', 'Store Name': 'Keokuk Spirits', 
'Address': '1013 MAIN', 'City': 'KEOKUK', 'Zip Code': '52632', 'Store Location': '1013 MAIN\nKEOKUK 52632\n(40.39978, -91.387531)', 
'County Number': '56', 'County': 'Lee', 'Category': '', 'Category Name': '', 'Vendor Number': '255', 'Vendor Name': 'Wilson Daniels Ltd.', 
'Item Number': '297', 'Item Description': 'Templeton Rye w/Flask', 'Pack': '6', 'Bottle Volume (ml)': '750', 'State Bottle Cost': '$18.09', 
'State Bottle Retail': '$27.14', 'Bottles Sold': '6', 'Sale (Dollars)': '$162.84', 'Volume Sold (Liters)': '4.50', 'Volume Sold (Gallons)': '1.19'}
'''